# Basic DNN

This is a simple attempt using a basic DNN. This will be used as a pre-trained model for my inference kernel. Some of the hyperparameters and choices are derived from my [EDA kernel](https://www.kaggle.com/yzgast/jsmp-basic-eda-starter). 

In [ ]:
import gc
import pickle

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from tensorflow import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

random.set_seed(5577)

## Data Loading

In [ ]:
trainDf = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

## Data Preparation

**Train / target split**

In [ ]:
trainDf = trainDf[trainDf["weight"] > 0]
trainDf.fillna(0, inplace=True)

dropCols = ["resp", "resp_1", "resp_2", "resp_3", "resp_4", "ts_id"]
dfTrain = trainDf.drop(columns=dropCols).values.astype(np.float32)
targets = (trainDf["resp"] > 0).astype(np.int).values

**PCA**

In [ ]:
scaler = MinMaxScaler()
scaledTrain = scaler.fit_transform(dfTrain)

pca = PCA(n_components=11)
pca.fit(scaledTrain)

pickledPCA = open("pcaFit.pkl", 'wb')
pickle.dump(pca, pickledPCA)
pickledPCA.close()

sTrainFeatures = pca.transform(scaledTrain)
print(sTrainFeatures.shape)
featuresCount = sTrainFeatures.shape[1]

## Cleaning variables

In [ ]:
del dfTrain
del pca
del scaledTrain
gc.collect()

## Hyperparameters

In [ ]:
nSplits = 5
batchSize = 50000
epochCount = 128
hiddenLayerSize = 1024
dropOut = 0.2

## Model Creation

In [ ]:
def getModel(hiddenLayerSize=1024, dropOut=0.2):
	model = Sequential()
	model.add(Dense(hiddenLayerSize, input_dim=featuresCount, activation='relu'))
	model.add(Dropout(dropOut))
	model.add(Dense(hiddenLayerSize,activation='relu'))
	model.add(Dropout(dropOut))
	model.add(Dense(hiddenLayerSize,activation='relu'))
	model.add(Dropout(dropOut))
	model.add(Dense(1, activation="sigmoid"))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
	return model


## Fitting the model

In [ ]:
models = []
history = {}
verbosity = 1

kfold = KFold(n_splits=nSplits, shuffle=True)
for j, (train_idx, val_idx) in enumerate(kfold.split(sTrainFeatures)):
	model = getModel(hiddenLayerSize, dropOut)
	history[j] = model.fit(sTrainFeatures[train_idx], targets[train_idx], validation_data=(sTrainFeatures[val_idx], targets[val_idx]), batch_size=batchSize, epochs=epochCount, verbose=verbosity)
	scores = model.evaluate(sTrainFeatures[val_idx], targets[val_idx], verbose=verbosity)
	print('Fold %d: %s of %f' % (j,model.metrics_names[0],scores[0]))
	print('Fold %d: %s of %f' % (j,model.metrics_names[1],scores[1]))
	models.append(model)


## Plotting accuracy & loss

In [ ]:
plt.figure(figsize=(15,7))
for k,v in history.items():
	plt.plot(v.history["accuracy"], color='#d13812', label="Accuracy Fold "+str(k))
	plt.plot(v.history["val_accuracy"], color='#6dbc1e', label="ValAccuracy Fold "+str(k))

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Folds Accuracy Compound')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
for k,v in history.items():
	plt.plot(v.history["loss"], color='#d13812', label="Loss Fold "+str(k))
	plt.plot(v.history["val_loss"], color='#6dbc1e', label="ValLoss Fold "+str(k))

plt.xlabel('Epochs')
plt.ylabel('Error')
plt.title('Folds Error Compound')
plt.legend()
plt.show()

## Saving the model

In [ ]:
model.save("model.h5")